In [17]:
# Install required packages if not already installed
# !pip install anywidget traitlets


In [18]:
%load_ext autoreload
%autoreload 2
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from experiments.notebooks.utils.metrics import fetch_metrics
from experiments.notebooks.utils.monitoring import monitor_training_statuses
from experiments.notebooks.utils.replays import show_replay
from experiments.notebooks.utils.training import launch_training
from experiments.notebooks.utils.metrics import find_training_jobs

%matplotlib inline
plt.style.use("default")

# Add utils directory to path
sys.path.append(os.path.join(os.getcwd(), 'utils'))

%load_ext anywidget

print("Setup complete! Auto-reload enabled.")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The anywidget extension is already loaded. To reload it, use:
  %reload_ext anywidget
Setup complete! Auto-reload enabled.


In [19]:
from experiments.notebooks.utils.heatmap_widget import HeatmapWidget, create_demo_heatmap, create_heatmap_widget

# Create a demo heatmap with sample data
demo_widget = create_demo_heatmap()

# Display the widget
demo_widget


🎯 Creating demo heatmap widget...
🚀 HeatmapWidget initialized successfully!
📊 Data set with 3 policies and 4 evaluations
📈 Selected metric: reward
✅ Demo heatmap widget created with sample data!


HeatmapWidget(heatmap_data={'cells': {'policy_alpha_v1': {'navigation/maze1': {'value': 85.2, 'replayUrl': 'sa…

In [20]:
import ipywidgets as widgets
from IPython.display import display

from experiments.notebooks.utils.heatmap_widget import HeatmapWidget, create_demo_heatmap, create_heatmap_widget

# Create a demo heatmap with sample data
demo_widget = create_demo_heatmap()
display(demo_widget)

w = widgets.Button(description="Click me", style=dict(width="200px", height="50px"))
display(w)
# Display the widget
print(demo_widget)

🎯 Creating demo heatmap widget...
🚀 HeatmapWidget initialized successfully!
📊 Data set with 3 policies and 4 evaluations
📈 Selected metric: reward
✅ Demo heatmap widget created with sample data!


HeatmapWidget(heatmap_data={'cells': {'policy_alpha_v1': {'navigation/maze1': {'value': 85.2, 'replayUrl': 'sa…

Button(description='Click me', style=ButtonStyle())

HeatmapWidget(heatmap_data={'cells': {'policy_alpha_v1': {'navigation/maze1': {'value': 85.2, 'replayUrl': 'sample_replay_1.json', 'evalName': 'navigation/maze1'}, 'navigation/maze2': {'value': 78.9, 'replayUrl': 'sample_replay_2.json', 'evalName': 'navigation/maze2'}, 'combat/arena1': {'value': 92.1, 'replayUrl': 'sample_replay_3.json', 'evalName': 'combat/arena1'}, 'combat/arena2': {'value': 88.7, 'replayUrl': 'sample_replay_4.json', 'evalName': 'combat/arena2'}}, 'policy_beta_v2': {'navigation/maze1': {'value': 79.8, 'replayUrl': 'sample_replay_5.json', 'evalName': 'navigation/maze1'}, 'navigation/maze2': {'value': 83.4, 'replayUrl': 'sample_replay_6.json', 'evalName': 'navigation/maze2'}, 'combat/arena1': {'value': 87.3, 'replayUrl': 'sample_replay_7.json', 'evalName': 'combat/arena1'}, 'combat/arena2': {'value': 91.2, 'replayUrl': 'sample_replay_8.json', 'evalName': 'combat/arena2'}}, 'policy_gamma_v1': {'navigation/maze1': {'value': 82.1, 'replayUrl': 'sample_replay_9.json', 'eva

In [21]:
# Create a new heatmap widget
custom_widget = create_heatmap_widget()

# Define your data structure
# This should match the format expected by the observatory dashboard
cells_data = {
    'my_policy_v1': {
        'task_a/level1': {
            'value': 95.0, 
            'replayUrl': 'https://example.com/replay1.json', 
            'evalName': 'task_a/level1'
        },
        'task_a/level2': {
            'value': 87.5, 
            'replayUrl': 'https://example.com/replay2.json', 
            'evalName': 'task_a/level2'
        },
        'task_b/challenge1': {
            'value': 92.3, 
            'replayUrl': 'https://example.com/replay3.json', 
            'evalName': 'task_b/challenge1'
        },
    },
    'my_policy_v2': {
        'task_a/level1': {
            'value': 88.2, 
            'replayUrl': 'https://example.com/replay4.json', 
            'evalName': 'task_a/level1'
        },
        'task_a/level2': {
            'value': 91.0, 
            'replayUrl': 'https://example.com/replay5.json', 
            'evalName': 'task_a/level2'
        },
        'task_b/challenge1': {
            'value': 89.7, 
            'replayUrl': 'https://example.com/replay6.json', 
            'evalName': 'task_b/challenge1'
        },
    },
}

eval_names = ['task_a/level1', 'task_a/level2', 'task_b/challenge1']
policy_names = ['my_policy_v1', 'my_policy_v2']
policy_averages = {
    'my_policy_v1': 91.6,
    'my_policy_v2': 89.6,
}

# Set the data
custom_widget.set_data(
    cells=cells_data,
    eval_names=eval_names,
    policy_names=policy_names,
    policy_average_scores=policy_averages,
    selected_metric="Custom Score (%)"
)

# Display the widget
custom_widget


🚀 HeatmapWidget initialized successfully!
📊 Data set with 2 policies and 3 evaluations
📈 Selected metric: Custom Score (%)


HeatmapWidget(heatmap_data={'cells': {'my_policy_v1': {'task_a/level1': {'value': 95.0, 'replayUrl': 'https://…

In [24]:
# Create another widget for callback demonstration
callback_widget = create_heatmap_widget()

# Set up the same data as before
callback_widget.set_data(
    cells=cells_data,
    eval_names=eval_names,
    policy_names=policy_names,
    policy_average_scores=policy_averages,
    selected_metric="Interactive Score (%)"
)

# Define callback functions
def handle_cell_selection(cell_info):
    """Called when user hovers over a cell (not 'overall' column)."""
    print(f"📍 Cell selected: {cell_info['policyUri']} on evaluation '{cell_info['evalName']}'")

def handle_replay_opened(replay_info):
    """Called when user double-clicks on a cell to open replay."""
    print(f"🎬 Replay opened: {replay_info['replayUrl']}")
    print(f"   Policy: {replay_info['policyUri']}")
    print(f"   Evaluation: {replay_info['evalName']}")

# Register the callbacks
callback_widget.on_cell_selected(handle_cell_selection)
callback_widget.on_replay_opened(handle_replay_opened)

# Display the widget
callback_widget


🚀 HeatmapWidget initialized successfully!
📊 Data set with 2 policies and 3 evaluations
📈 Selected metric: Interactive Score (%)


HeatmapWidget(heatmap_data={'cells': {'my_policy_v1': {'task_a/level1': {'value': 95.0, 'replayUrl': 'https://…

In [26]:
import ipywidgets as widgets

slider = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

slider

def on_slider_change(change):
    print(change)

slider.observe(on_slider_change, names='value')

demo_widget

HeatmapWidget(heatmap_data={'cells': {'policy_alpha_v1': {'navigation/maze1': {'value': 85.2, 'replayUrl': 'sa…